## 下載檔案

In [7]:
import re 
import pandas as pd 
import pdfplumber as pr
import requests
# 設置 PDF 檔案的下載鏈接和檔案名稱
company_code = "2317"
year = "110"

url = "https://mops.twse.com.tw/server-java/FileDownLoad?step=9&filePath=/home/html/nas/protect/t100/&fileName=t100sa11_" + company_code + "_" + year + ".pdf"
filename = "t100sa11_" + company_code + "_" + year + ".pdf"

# 發送 GET 請求並下載檔案
response = requests.get(url)
with open(filename, "wb") as f:
    f.write(response.content)


## 另一種擷取方式，先判斷哪幾頁有準則，再擷取那幾頁

In [2]:
pdf = pr.open(filename)
ps = pdf.pages
df_new=[]
new_page=[]
columns_count = []
 # 取得總頁數
total_pages = len(pdf.pages)
# 設定起始頁數
start_page = int(total_pages * 0.8)
    
# 迭代每頁，從設定的起始頁開始
for page in pdf.pages[start_page:]:
    text = page.extract_text()
    if ("GRI" and "頁" )in text:
        new_page.append(page)
        for page in new_page:
            tables = page.extract_tables()
            for df in tables:
                df1= pd.DataFrame(df, columns=df[0])
                df1=df1.drop_duplicates()
                df_new.append(df1) 

In [1]:
UR=[]
final=[]
pattern="\d{1,3}[\-－]\d{1,2}"

for df in df_new:
    if df.shape[1] >= 4:
        final.append(df)
        for df in final:
            # 检查当前DataFrame对象的前两列是否与正则表达式匹配
            
            col1 = df.iloc[:, 0]
            col2 = df.iloc[:, 1]
            
            matches = col1.str.contains(pattern) | col2.str.contains(pattern)

            # 如果至少有一个单元格与正则表达式匹配，则将该DataFrame添加到UR列表中
            if matches.any():
                UR.append(df)
UR

NameError: name 'df_new' is not defined

In [ ]:
# 對於每個DataFrame，只保留包含指定模式的列
new_df=pd.DataFrame()
for df in UR:
    pattern = r"\d{1,3}[\-－]\d{1,2}"
    has_page_condition = lambda x: x.str.contains("頁")
    # 使用 str.contains() 方法和 `|` 操作符組合篩選條件
    filtered_df = df.loc[:, df.apply(lambda x: x.str.contains(pattern) | has_page_condition(x)).any()]
    new_df = pd.concat([new_df, filtered_df], axis=0)

## 輸出表格為.XLSX

In [12]:
new_df=new_df.reset_index(drop=True)
new_df=new_df.drop_duplicates()
new_df=new_df.rename(columns={new_df.columns[1]:"pages"})
output_excel=filename+'.xlsx'
new_df.to_excel(output_excel,encoding="utf-8")

## 讀取整理好之excel表格頁碼切割pdf並貼回路徑

In [4]:
# Read in the Excel file as a dataframe

import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl import load_workbook
from pdfrw import PdfReader, PdfWriter


df = pd.read_excel(output_excel, engine='openpyxl')
df

# Loop through each row in the DataFrame
for i in range(df.shape[0]):
    # Split the pages string into individual page numbers
    pages = list(map(int, df.loc[i, 'pages'].split(',')))
    
    # Generate the corresponding PDF file
    output_file = os.path.join("C:/Users/pojid/Capstone/output", str(i+1) + '.pdf')
    writer = PdfWriter()
    for page in pages:
        writer.addpage(PdfReader(filename).pages[page-1])
    writer.write(output_file)
    
    # Update the pdf_path column in the DataFrame
    df.loc[i, 'pdf_path'] = output_file


In [6]:
# create a Pandas Excel writer using OpenPyXL as the engine
writer = pd.ExcelWriter(output_excel, engine='openpyxl')

# write the DataFrame to the Excel file
df.to_excel(writer, index=False)

# save the changes to the file
writer.save()

C:\Users\pojid\AppData\Local\Temp\ipykernel_16004\3778995600.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
